In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# InputResidsComp

`InputResidsComp` is a specialized implementation of [ImplicitComponent](../../core_features/working_with_components/implicit_component.ipynb) that is intended to allow the user to simply add residuals to a system by treating any inputs to the components as the value of the associated residual.

Unlike `BalanceComp`, implicit outputs do not map one-to-one with the inputs. That is, the number of output variables for `InputResidsComp` does not have to match the number of input variables, but **the total size of the inputs and outputs must be the same**.

`InputResidsComp` can make it easier to convert an MDO problem from a "SAND" (simultaneous analysis and design) formumlation to an "MDF" (multiple design feasible) formulation by adding zero-valued equality constraints to it as inputs, and the associated design variables to it as implicit outputs.

## InputResidsComp Constructor

The call signature for the `InputResidsComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.input_resids_comp.InputResidsComp.__init__()
        :noindex:
```

## Example:  Single state vector, multiple equations of constraint

The following example uses a InputResidsComp to implicitly solve the
equations:

\begin{align}
    x_0 + x_1 &= 5 \\
    x_2 + x_3 &= 10 \\
    x_0 &= x_3 \\
    \left\Vert \bar{x} \right\Vert &= 9
\end{align}


In [ ]:
import openmdao.api as om

prob = om.Problem()

bal = om.BalanceComp()
bal.add_balance('x', use_mult=True)

exec_comp = om.ExecComp(['y[0]=x[0] + x[1] - 5',
                         'y[1]=x[2] + x[3] - 10',
                         'y[2]=x[0] - x[3]',
                         'z=dot(x, x)**0.5 - 9'],
                        x={'shape': (4,)},
                        y={'val': [1., 1., 1.]},
                        z={'val': 2.})

prob.model.add_subsystem(name='exec', subsys=exec_comp)
resids = prob.model.add_subsystem(name='resids', subsys=om.InputResidsComp())

resids.add_output('x', shape_by_conn=True)
resids.add_input('res_0', shape_by_conn=True)
resids.add_input('res_1', shape_by_conn=True)

prob.model.connect('resids.x', 'exec.x')
prob.model.connect('exec.y', 'resids.res_0')
prob.model.connect('exec.z', 'resids.res_1')

prob.model.linear_solver = om.DirectSolver(assemble_jac=True)
prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False, maxiter=100, iprint=0)
prob.set_solver_print(2)


prob.setup()

prob.set_val('resids.x', [1., 1., 10, 5])

prob.final_setup()

prob.run_model()

prob.model.list_vars(print_arrays=True);
